# setPath

In [ ]:
from sys import path
path.append(r"../")

# fourCFR

In [ ]:
import os
import argparse
import pandas as pd
import numpy as np
import torch

from utils import log, CausalDataset
from module.SynCBIV import run as run_SynCBIV

os.environ["CUDA_VISIBLE_DEVICES"] = '1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

def get_args():
    argparser = argparse.ArgumentParser(description=__doc__)
    # About run setting !!!!
    argparser.add_argument('--seed',default=2021,type=int,help='The random seed')
    argparser.add_argument('--mode',default='vx',type=str,help='The choice of v/x/vx/xx')
    argparser.add_argument('--rewrite_log',default=False,type=bool,help='Whether rewrite log file')
    argparser.add_argument('--use_gpu',default=True,type=bool,help='The use of GPU')
    # About data setting ~~~~
    argparser.add_argument('--num',default=10000,type=int,help='The num of train\val\test dataset')
    argparser.add_argument('--num_reps',default=10,type=int,help='The num of train\val\test dataset')
    argparser.add_argument('--ate',default=0,type=float,help='The ate of constant')
    argparser.add_argument('--sc',default=1,type=float,help='The sc')
    argparser.add_argument('--sh',default=0,type=float,help='The sh')
    argparser.add_argument('--one',default=1,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--depX',default=0.05,type=float,help='Whether generates harder datasets')
    argparser.add_argument('--depU',default=0.05,type=float,help='Whether generates harder datasets')
    argparser.add_argument('--VX',default=1,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--mV',default=2,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--mX',default=4,type=int,help='The dim of Confounding variables X')
    argparser.add_argument('--mU',default=4,type=int,help='The dim of Unobserved confounding variables U')
    argparser.add_argument('--storage_path',default='../Data/',type=str,help='The dir of data storage')
    # Syn
    argparser.add_argument('--syn_alpha',default=0.01,type=float,help='')
    argparser.add_argument('--syn_lambda',default=0.0001,type=float,help='')
    argparser.add_argument('--syn_twoStage',default=True,type=bool,help='')
    # About Debug or Show
    argparser.add_argument('--verbose',default=1,type=int,help='The level of verbose')
    argparser.add_argument('--epoch_show',default=5,type=int,help='The epochs of show time')
    args = argparser.parse_args(args=[])
    return args

args = get_args()

if args.use_gpu:
    device = torch.device('cuda' if torch.cuda.is_available() and args.use_gpu else "cpu")
    
# set path
which_benchmark = 'Syn_'+'_'.join(str(item) for item in [args.sc, args.sh, args.one, args.depX, args.depU,args.VX])
which_dataset = '_'.join(str(item) for item in [args.mV, args.mX, args.mU])
resultDir = args.storage_path + f'/results/{which_benchmark}_{which_dataset}/{args.num}/'
dataDir = f'{args.storage_path}/data/{which_benchmark}/{which_dataset}/'
os.makedirs(os.path.dirname(resultDir), exist_ok=True)
logfile = f'{resultDir}/log.txt'

if args.rewrite_log:
    f = open(logfile,'w')
    f.close()

results = []
alpha = args.syn_alpha
for exp in range(args.num_reps):
    # load data
    train_df = pd.read_csv(dataDir + f'{exp}/{args.mode}/train.csv')[:args.num]
    val_df = pd.read_csv(dataDir + f'{exp}/{args.mode}/val.csv')[:args.num]
    test_df = pd.read_csv(dataDir + f'{exp}/{args.mode}/test.csv')[:args.num]

    train = CausalDataset(train_df, variables = ['u','x','v','z','p','s','m','t','g','y','f','c'])
    val = CausalDataset(val_df, variables = ['u','x','v','z','p','s','m','t','g','y','f','c'])
    test = CausalDataset(test_df, variables = ['u','x','v','z','p','s','m','t','g','y','f','c'])

    res_list = []
    
    args.syn_twoStage = False
    args.syn_alpha = 0
    mse_val, obj_val, final = run_SynCBIV(exp, args, dataDir, resultDir, train, val, test, device)
    res_list = res_list + [obj_val['ate_train'],obj_val['ate_test']]
    
    args.syn_twoStage = False
    args.syn_alpha = alpha
    mse_val, obj_val, final = run_SynCBIV(exp, args, dataDir, resultDir, train, val, test, device)
    res_list = res_list + [obj_val['ate_train'],obj_val['ate_test']]
    
    args.syn_twoStage = True
    args.syn_alpha = 0
    mse_val, obj_val, final = run_SynCBIV(exp, args, dataDir, resultDir, train, val, test, device)
    res_list = res_list + [obj_val['ate_train'],obj_val['ate_test']]
    
    args.syn_twoStage = True
    args.syn_alpha = alpha
    mse_val, obj_val, final = run_SynCBIV(exp, args, dataDir, resultDir, train, val, test, device)
    res_list = res_list + [obj_val['ate_train'],obj_val['ate_test']]
    
    res = np.array(res_list) - 1.0
    results.append(res)

results.append(np.mean(results,0))
results.append(np.std(results,0))
    
res_df = pd.DataFrame(np.array(results),
                       columns=[ alpha+data_cls for alpha in ['Direct', 'CFR', 'TwoStage', 'SynCBIV'] for data_cls in ['_train', '_test']]).round(4)
res_df.to_csv(resultDir + f'SynCBIV_{args.mode}_result.csv', index=False)
res_df


# DRCFR

In [ ]:
import os
import argparse
import pandas as pd
import numpy as np
import torch

from utils import log, CausalDataset
from module.DRCFR import run as run_DRCFR

os.environ["CUDA_VISIBLE_DEVICES"] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

def get_args():
    argparser = argparse.ArgumentParser(description=__doc__)
    # About run setting !!!!
    argparser.add_argument('--seed',default=2021,type=int,help='The random seed')
    argparser.add_argument('--mode',default='vx',type=str,help='The choice of v/x/vx/xx')
    argparser.add_argument('--rewrite_log',default=False,type=bool,help='Whether rewrite log file')
    argparser.add_argument('--use_gpu',default=True,type=bool,help='The use of GPU')
    # About data setting ~~~~
    argparser.add_argument('--num',default=10000,type=int,help='The num of train\val\test dataset')
    argparser.add_argument('--num_reps',default=10,type=int,help='The num of train\val\test dataset')
    argparser.add_argument('--ate',default=0,type=float,help='The ate of constant')
    argparser.add_argument('--sc',default=1,type=float,help='The sc')
    argparser.add_argument('--sh',default=0,type=float,help='The sh')
    argparser.add_argument('--one',default=1,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--depX',default=0.05,type=float,help='Whether generates harder datasets')
    argparser.add_argument('--depU',default=0.05,type=float,help='Whether generates harder datasets')
    argparser.add_argument('--VX',default=1,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--mV',default=2,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--mX',default=4,type=int,help='The dim of Confounding variables X')
    argparser.add_argument('--mU',default=4,type=int,help='The dim of Unobserved confounding variables U')
    argparser.add_argument('--storage_path',default='../Data/',type=str,help='The dir of data storage')
    # DRCFR
    argparser.add_argument('--drcfr_loss',default='l2',type=str,help='')
    argparser.add_argument('--drcfr_alpha',default=0.01,type=float,help='')
    argparser.add_argument('--drcfr_beta',default=0.01,type=float,help='')
    argparser.add_argument('--drcfr_lambda',default=0.0001,type=float,help='')
    # About Debug or Show
    argparser.add_argument('--verbose',default=1,type=int,help='The level of verbose')
    argparser.add_argument('--epoch_show',default=5,type=int,help='The epochs of show time')
    args = argparser.parse_args(args=[])
    return args

args = get_args()

if args.use_gpu:
    device = torch.device('cuda' if torch.cuda.is_available() and args.use_gpu else "cpu")
    
# set path
which_benchmark = 'Syn_'+'_'.join(str(item) for item in [args.sc, args.sh, args.one, args.depX, args.depU,args.VX])
which_dataset = '_'.join(str(item) for item in [args.mV, args.mX, args.mU])
resultDir = args.storage_path + f'/results/{which_benchmark}_{which_dataset}/{args.num}/'
dataDir = f'{args.storage_path}/data/{which_benchmark}/{which_dataset}/'
os.makedirs(os.path.dirname(resultDir), exist_ok=True)
logfile = f'{resultDir}/log.txt'

if args.rewrite_log:
    f = open(logfile,'w')
    f.close()

results = []
for exp in range(args.num_reps):
    # load data
    train_df = pd.read_csv(dataDir + f'{exp}/{args.mode}/train.csv')[:args.num]
    val_df = pd.read_csv(dataDir + f'{exp}/{args.mode}/val.csv')[:args.num]
    test_df = pd.read_csv(dataDir + f'{exp}/{args.mode}/test.csv')[:args.num]

    train = CausalDataset(train_df, variables = ['u','x','v','z','p','s','m','t','g','y','f','c'])
    val = CausalDataset(val_df, variables = ['u','x','v','z','p','s','m','t','g','y','f','c'])
    test = CausalDataset(test_df, variables = ['u','x','v','z','p','s','m','t','g','y','f','c'])

    res_list = []
    
    mse_val, obj_val, final = run_DRCFR(exp, args, dataDir, resultDir, train, val, test, device)
    res_list = res_list + [obj_val['ate_train'],obj_val['ate_test']]
    
    res = np.array(res_list) - 1.0
    results.append(res)

results.append(np.mean(results,0))
results.append(np.std(results,0))
    
res_df = pd.DataFrame(np.array(results),
                       columns=[ alpha+data_cls for alpha in ['DRCFR'] for data_cls in ['_train', '_test']]).round(4)
res_df.to_csv(resultDir + f'DRCFR_{args.mode}_result.csv', index=False)
res_df

# DeepIV

In [ ]:
import os
import argparse
import pandas as pd
import numpy as np
import torch

from utils import log, CausalDataset
from module.DeepIV import run as run_DeepIV

os.environ["CUDA_VISIBLE_DEVICES"] = '2'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

def get_args():
    argparser = argparse.ArgumentParser(description=__doc__)
    # About run setting !!!!
    argparser.add_argument('--seed',default=2021,type=int,help='The random seed')
    argparser.add_argument('--mode',default='vx',type=str,help='The choice of v/x/vx/xx')
    argparser.add_argument('--rewrite_log',default=False,type=bool,help='Whether rewrite log file')
    argparser.add_argument('--use_gpu',default=True,type=bool,help='The use of GPU')
    # About data setting ~~~~
    argparser.add_argument('--num',default=10000,type=int,help='The num of train\val\test dataset')
    argparser.add_argument('--num_reps',default=10,type=int,help='The num of train\val\test dataset')
    argparser.add_argument('--ate',default=0,type=float,help='The ate of constant')
    argparser.add_argument('--sc',default=1,type=float,help='The sc')
    argparser.add_argument('--sh',default=0,type=float,help='The sh')
    argparser.add_argument('--one',default=1,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--depX',default=0.05,type=float,help='Whether generates harder datasets')
    argparser.add_argument('--depU',default=0.05,type=float,help='Whether generates harder datasets')
    argparser.add_argument('--VX',default=1,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--mV',default=2,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--mX',default=4,type=int,help='The dim of Confounding variables X')
    argparser.add_argument('--mU',default=4,type=int,help='The dim of Unobserved confounding variables U')
    argparser.add_argument('--storage_path',default='../Data/',type=str,help='The dir of data storage')
    # About DeepIV
    argparser.add_argument('--deepiv_t_loss',default='binary_crossentropy',type=str,help='mse/binary_crossentropy/mae/mixture_of_gaussians')
    argparser.add_argument('--deepiv_y_loss',default='mse',type=str,help='mse/binary_crossentropy/mae/mixture_of_gaussians')
    argparser.add_argument('--deepiv_batch_size',default=100,type=int,help='The size of one batch')
    argparser.add_argument('--deepiv_n_components',default=10,type=int,help='n_components')
    argparser.add_argument('--deepiv_hidden',default=[128, 64, 32],type=list,help='hidden_layers')
    argparser.add_argument('--deepiv_act',default="relu",type=str,help='act')
    # About Debug or Show
    argparser.add_argument('--verbose',default=1,type=int,help='The level of verbose')
    argparser.add_argument('--epoch_show',default=5,type=int,help='The epochs of show time')
    args = argparser.parse_args(args=[])
    return args

args = get_args()

if args.use_gpu:
    device = torch.device('cuda' if torch.cuda.is_available() and args.use_gpu else "cpu")
    
# set path
which_benchmark = 'Syn_'+'_'.join(str(item) for item in [args.sc, args.sh, args.one, args.depX, args.depU,args.VX])
which_dataset = '_'.join(str(item) for item in [args.mV, args.mX, args.mU])
resultDir = args.storage_path + f'/results/{which_benchmark}_{which_dataset}/{args.num}/'
dataDir = f'{args.storage_path}/data/{which_benchmark}/{which_dataset}/'
os.makedirs(os.path.dirname(resultDir), exist_ok=True)
logfile = f'{resultDir}/log.txt'

if args.rewrite_log:
    f = open(logfile,'w')
    f.close()

results = []
for exp in range(args.num_reps):
    # load data
    train_df = pd.read_csv(dataDir + f'{exp}/{args.mode}/train.csv')[:args.num]
    val_df = pd.read_csv(dataDir + f'{exp}/{args.mode}/val.csv')[:args.num]
    test_df = pd.read_csv(dataDir + f'{exp}/{args.mode}/test.csv')[:args.num]

    train = CausalDataset(train_df, variables = ['u','x','v','z','p','s','m','t','g','y','f','c'])
    val = CausalDataset(val_df, variables = ['u','x','v','z','p','s','m','t','g','y','f','c'])
    test = CausalDataset(test_df, variables = ['u','x','v','z','p','s','m','t','g','y','f','c'])

    pred_ate_train, pred_ate_test = run_DeepIV(exp, args, dataDir, resultDir, train, val, test, device)
    res_list = [pred_ate_train, pred_ate_test]
    
    res = np.array(res_list) - 1.0
    results.append(res)

results.append(np.mean(results,0))
results.append(np.std(results,0))
    
res_df = pd.DataFrame(np.array(results),
                       columns=[ alpha+data_cls for alpha in ['DeepIV'] for data_cls in ['_train', '_test']]).round(4)
res_df.to_csv(resultDir + f'DeepIV_{args.mode}_result.csv', index=False)
res_df

# DeepIV-gmm

In [ ]:
import os
import argparse
import pandas as pd
import numpy as np
import torch

from utils import log, CausalDataset
from module.DeepIV import run as run_DeepIV

os.environ["CUDA_VISIBLE_DEVICES"] = '2'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

def get_args():
    argparser = argparse.ArgumentParser(description=__doc__)
    # About run setting !!!!
    argparser.add_argument('--seed',default=2021,type=int,help='The random seed')
    argparser.add_argument('--mode',default='vx',type=str,help='The choice of v/x/vx/xx')
    argparser.add_argument('--rewrite_log',default=False,type=bool,help='Whether rewrite log file')
    argparser.add_argument('--use_gpu',default=True,type=bool,help='The use of GPU')
    # About data setting ~~~~
    argparser.add_argument('--num',default=10000,type=int,help='The num of train\val\test dataset')
    argparser.add_argument('--num_reps',default=10,type=int,help='The num of train\val\test dataset')
    argparser.add_argument('--ate',default=0,type=float,help='The ate of constant')
    argparser.add_argument('--sc',default=1,type=float,help='The sc')
    argparser.add_argument('--sh',default=0,type=float,help='The sh')
    argparser.add_argument('--one',default=1,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--depX',default=0.05,type=float,help='Whether generates harder datasets')
    argparser.add_argument('--depU',default=0.05,type=float,help='Whether generates harder datasets')
    argparser.add_argument('--VX',default=1,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--mV',default=2,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--mX',default=4,type=int,help='The dim of Confounding variables X')
    argparser.add_argument('--mU',default=4,type=int,help='The dim of Unobserved confounding variables U')
    argparser.add_argument('--storage_path',default='../Data/',type=str,help='The dir of data storage')
    # About DeepIV
    argparser.add_argument('--deepiv_t_loss',default='mixture_of_gaussians',type=str,help='mse/binary_crossentropy/mae/mixture_of_gaussians')
    argparser.add_argument('--deepiv_y_loss',default='mse',type=str,help='mse/binary_crossentropy/mae/mixture_of_gaussians')
    argparser.add_argument('--deepiv_batch_size',default=100,type=int,help='The size of one batch')
    argparser.add_argument('--deepiv_n_components',default=10,type=int,help='n_components')
    argparser.add_argument('--deepiv_hidden',default=[128, 64, 32],type=list,help='hidden_layers')
    argparser.add_argument('--deepiv_act',default="relu",type=str,help='act')
    # About Debug or Show
    argparser.add_argument('--verbose',default=1,type=int,help='The level of verbose')
    argparser.add_argument('--epoch_show',default=5,type=int,help='The epochs of show time')
    args = argparser.parse_args(args=[])
    return args

args = get_args()

if args.use_gpu:
    device = torch.device('cuda' if torch.cuda.is_available() and args.use_gpu else "cpu")
    
# set path
which_benchmark = 'Syn_'+'_'.join(str(item) for item in [args.sc, args.sh, args.one, args.depX, args.depU,args.VX])
which_dataset = '_'.join(str(item) for item in [args.mV, args.mX, args.mU])
resultDir = args.storage_path + f'/results/{which_benchmark}_{which_dataset}/{args.num}/'
dataDir = f'{args.storage_path}/data/{which_benchmark}/{which_dataset}/'
os.makedirs(os.path.dirname(resultDir), exist_ok=True)
logfile = f'{resultDir}/log.txt'

if args.rewrite_log:
    f = open(logfile,'w')
    f.close()

results = []
for exp in range(args.num_reps):
    # load data
    train_df = pd.read_csv(dataDir + f'{exp}/{args.mode}/train.csv')[:args.num]
    val_df = pd.read_csv(dataDir + f'{exp}/{args.mode}/val.csv')[:args.num]
    test_df = pd.read_csv(dataDir + f'{exp}/{args.mode}/test.csv')[:args.num]

    train = CausalDataset(train_df, variables = ['u','x','v','z','p','s','m','t','g','y','f','c'])
    val = CausalDataset(val_df, variables = ['u','x','v','z','p','s','m','t','g','y','f','c'])
    test = CausalDataset(test_df, variables = ['u','x','v','z','p','s','m','t','g','y','f','c'])

    pred_ate_train, pred_ate_test = run_DeepIV(exp, args, dataDir, resultDir, train, val, test, device)
    res_list = [pred_ate_train, pred_ate_test]
    
    res = np.array(res_list) - 1.0
    results.append(res)

results.append(np.mean(results,0))
results.append(np.std(results,0))
    
res_df = pd.DataFrame(np.array(results),
                       columns=[ alpha+data_cls for alpha in ['DeepIV_gmm'] for data_cls in ['_train', '_test']]).round(4)
res_df.to_csv(resultDir + f'DeepIV_gmm_{args.mode}_result.csv', index=False)
res_df

# OneSIV

In [ ]:
import os
import argparse
import pandas as pd
import numpy as np
import torch

from utils import log, CausalDataset
from module.OneSIV import run as run_OneSIV

os.environ["CUDA_VISIBLE_DEVICES"] = '1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

def get_args():
    argparser = argparse.ArgumentParser(description=__doc__)
    # About run setting !!!!
    argparser.add_argument('--seed',default=2021,type=int,help='The random seed')
    argparser.add_argument('--mode',default='vx',type=str,help='The choice of v/x/vx/xx')
    argparser.add_argument('--rewrite_log',default=False,type=bool,help='Whether rewrite log file')
    argparser.add_argument('--use_gpu',default=True,type=bool,help='The use of GPU')
    # About data setting ~~~~
    argparser.add_argument('--num',default=10000,type=int,help='The num of train\val\test dataset')
    argparser.add_argument('--num_reps',default=10,type=int,help='The num of train\val\test dataset')
    argparser.add_argument('--ate',default=0,type=float,help='The ate of constant')
    argparser.add_argument('--sc',default=1,type=float,help='The sc')
    argparser.add_argument('--sh',default=0,type=float,help='The sh')
    argparser.add_argument('--one',default=1,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--depX',default=0.05,type=float,help='Whether generates harder datasets')
    argparser.add_argument('--depU',default=0.05,type=float,help='Whether generates harder datasets')
    argparser.add_argument('--VX',default=1,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--mV',default=2,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--mX',default=4,type=int,help='The dim of Confounding variables X')
    argparser.add_argument('--mU',default=4,type=int,help='The dim of Unobserved confounding variables U')
    argparser.add_argument('--storage_path',default='../Data/',type=str,help='The dir of data storage')
    # About OneSIV
    argparser.add_argument('--onesiv_loss',default='l2',type=str,help='The param of term 2')
    argparser.add_argument('--onesiv_lr',default=0.01,type=float,help='The param of term 1')
    argparser.add_argument('--onesiv_beta1',default=0.9,type=float,help='The param of term 1')
    argparser.add_argument('--onesiv_beta2',default=0.999,type=float,help='The param of term 1')
    argparser.add_argument('--onesiv_eps',default=1e-8,type=float,help='The param of term 1')
    argparser.add_argument('--onesiv_epochs',default=100,type=int,help='The size of one batch')
    argparser.add_argument('--onesiv_batch_size',default=128,type=int,help='The size of one batch')
    argparser.add_argument('--onesiv_dropout',default=0.5,type=float,help='The param of term 1')
    argparser.add_argument('--onesiv_w1',default=0.0017,type=float,help='The param of term 1')
    argparser.add_argument('--onesiv_w2',default=1.0,type=float,help='The param of term 2')
    # About Debug or Show
    argparser.add_argument('--verbose',default=1,type=int,help='The level of verbose')
    argparser.add_argument('--epoch_show',default=5,type=int,help='The epochs of show time')
    args = argparser.parse_args(args=[])
    return args

args = get_args()

if args.use_gpu:
    device = torch.device('cuda' if torch.cuda.is_available() and args.use_gpu else "cpu")
    
# set path
which_benchmark = 'Syn_'+'_'.join(str(item) for item in [args.sc, args.sh, args.one, args.depX, args.depU,args.VX])
which_dataset = '_'.join(str(item) for item in [args.mV, args.mX, args.mU])
resultDir = args.storage_path + f'/results/{which_benchmark}_{which_dataset}/{args.num}/'
dataDir = f'{args.storage_path}/data/{which_benchmark}/{which_dataset}/'
os.makedirs(os.path.dirname(resultDir), exist_ok=True)
logfile = f'{resultDir}/log.txt'

if args.rewrite_log:
    f = open(logfile,'w')
    f.close()

results = []
for exp in range(args.num_reps):
    # load data
    train_df = pd.read_csv(dataDir + f'{exp}/{args.mode}/train.csv')[:args.num]
    val_df = pd.read_csv(dataDir + f'{exp}/{args.mode}/val.csv')[:args.num]
    test_df = pd.read_csv(dataDir + f'{exp}/{args.mode}/test.csv')[:args.num]

    train = CausalDataset(train_df, variables = ['u','x','v','z','p','s','m','t','g','y','f','c'])
    val = CausalDataset(val_df, variables = ['u','x','v','z','p','s','m','t','g','y','f','c'])
    test = CausalDataset(test_df, variables = ['u','x','v','z','p','s','m','t','g','y','f','c'])

    res_list = []
    
    mse_ate, obj_ate, res20_ate, final_ate = run_OneSIV(exp, args, dataDir, resultDir, train, val, test, device)
    res_list = res_list + mse_ate + obj_ate + res20_ate + final_ate
    
    res = np.array(res_list) - 1.0
    results.append(res)

results.append(np.mean(results,0))
results.append(np.std(results,0))
    
res_df = pd.DataFrame(np.array(results),
                       columns=[ alpha+data_cls for alpha in ['MSE', 'Loss', 'Epoch20', 'Final'] for data_cls in ['_train', '_test']]).round(4)
res_df.to_csv(resultDir + f'OneSIV_{args.mode}_result.csv', index=False)
res_df

# DFL

In [ ]:
import os
import argparse
import pandas as pd
import numpy as np
import torch

from utils import log, CausalDataset
from module.DFL import run as run_DFL

os.environ["CUDA_VISIBLE_DEVICES"] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

def get_args():
    argparser = argparse.ArgumentParser(description=__doc__)
    # About run setting !!!!
    argparser.add_argument('--seed',default=2021,type=int,help='The random seed')
    argparser.add_argument('--mode',default='vx',type=str,help='The choice of v/x/vx/xx')
    argparser.add_argument('--rewrite_log',default=False,type=bool,help='Whether rewrite log file')
    argparser.add_argument('--use_gpu',default=True,type=bool,help='The use of GPU')
    # About data setting ~~~~
    argparser.add_argument('--num',default=10000,type=int,help='The num of train\val\test dataset')
    argparser.add_argument('--num_reps',default=10,type=int,help='The num of train\val\test dataset')
    argparser.add_argument('--ate',default=0,type=float,help='The ate of constant')
    argparser.add_argument('--sc',default=1,type=float,help='The sc')
    argparser.add_argument('--sh',default=0,type=float,help='The sh')
    argparser.add_argument('--one',default=1,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--depX',default=0.05,type=float,help='Whether generates harder datasets')
    argparser.add_argument('--depU',default=0.05,type=float,help='Whether generates harder datasets')
    argparser.add_argument('--VX',default=1,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--mV',default=2,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--mX',default=4,type=int,help='The dim of Confounding variables X')
    argparser.add_argument('--mU',default=4,type=int,help='The dim of Unobserved confounding variables U')
    argparser.add_argument('--storage_path',default='../Data/',type=str,help='The dir of data storage')
    # DFL
    argparser.add_argument('--dfl_t_loss',default='bin',type=str,help='mse/bin')
    argparser.add_argument('--dfl_y_loss',default='mse',type=str,help='mse/bin')
    # About Debug or Show
    argparser.add_argument('--verbose',default=1,type=int,help='The level of verbose')
    argparser.add_argument('--epoch_show',default=5,type=int,help='The epochs of show time')
    args = argparser.parse_args(args=[])
    return args

args = get_args()

if args.use_gpu:
    device = torch.device('cuda' if torch.cuda.is_available() and args.use_gpu else "cpu")
    
# set path
which_benchmark = 'Syn_'+'_'.join(str(item) for item in [args.sc, args.sh, args.one, args.depX, args.depU,args.VX])
which_dataset = '_'.join(str(item) for item in [args.mV, args.mX, args.mU])
resultDir = args.storage_path + f'/results/{which_benchmark}_{which_dataset}/{args.num}/'
dataDir = f'{args.storage_path}/data/{which_benchmark}/{which_dataset}/'
os.makedirs(os.path.dirname(resultDir), exist_ok=True)
logfile = f'{resultDir}/log.txt'

if args.rewrite_log:
    f = open(logfile,'w')
    f.close()

results = []
for exp in range(args.num_reps):
    # load data
    train_df = pd.read_csv(dataDir + f'{exp}/{args.mode}/train.csv')[:args.num]
    val_df = pd.read_csv(dataDir + f'{exp}/{args.mode}/val.csv')[:args.num]
    test_df = pd.read_csv(dataDir + f'{exp}/{args.mode}/test.csv')[:args.num]

    train = CausalDataset(train_df, variables = ['u','x','v','z','p','s','m','t','g','y','f','c'])
    val = CausalDataset(val_df, variables = ['u','x','v','z','p','s','m','t','g','y','f','c'])
    test = CausalDataset(test_df, variables = ['u','x','v','z','p','s','m','t','g','y','f','c'])

    res_list = []
    
    monitor = run_DFL(exp, args, dataDir, resultDir, train, val, test, device)
    res_list = res_list + [monitor.pred_ate_train_best.item(),monitor.pred_ate_test_best.item(),monitor.pred_ate_train_final.item(),monitor.pred_ate_test_final.item()]
    
    res = np.array(res_list) - 1.0
    results.append(res)

results.append(np.mean(results,0))
results.append(np.std(results,0))
    
res_df = pd.DataFrame(np.array(results),
                       columns=[ alpha+data_cls for alpha in ['best', 'final'] for data_cls in ['_train', '_test']]).round(4)
res_df.to_csv(resultDir + f'DFL_{args.mode}_result.csv', index=False)
res_df

# DFIV

In [ ]:
import os
import argparse
import pandas as pd
import numpy as np
import torch

from utils import log, CausalDataset
from module.DFIV import run as run_DFIV

os.environ["CUDA_VISIBLE_DEVICES"] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

def get_args():
    argparser = argparse.ArgumentParser(description=__doc__)
    # About run setting !!!!
    argparser.add_argument('--seed',default=2021,type=int,help='The random seed')
    argparser.add_argument('--mode',default='vx',type=str,help='The choice of v/x/vx/xx')
    argparser.add_argument('--rewrite_log',default=False,type=bool,help='Whether rewrite log file')
    argparser.add_argument('--use_gpu',default=True,type=bool,help='The use of GPU')
    # About data setting ~~~~
    argparser.add_argument('--num',default=10000,type=int,help='The num of train\val\test dataset')
    argparser.add_argument('--num_reps',default=10,type=int,help='The num of train\val\test dataset')
    argparser.add_argument('--ate',default=0,type=float,help='The ate of constant')
    argparser.add_argument('--sc',default=1,type=float,help='The sc')
    argparser.add_argument('--sh',default=0,type=float,help='The sh')
    argparser.add_argument('--one',default=1,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--depX',default=0.05,type=float,help='Whether generates harder datasets')
    argparser.add_argument('--depU',default=0.05,type=float,help='Whether generates harder datasets')
    argparser.add_argument('--VX',default=1,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--mV',default=2,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--mX',default=4,type=int,help='The dim of Confounding variables X')
    argparser.add_argument('--mU',default=4,type=int,help='The dim of Unobserved confounding variables U')
    argparser.add_argument('--storage_path',default='../Data/',type=str,help='The dir of data storage')
    # DFIV
    argparser.add_argument('--dfiv_t_loss',default='bin',type=str,help='mse/bin')
    argparser.add_argument('--dfiv_y_loss',default='mse',type=str,help='mse/bin')
    # About Debug or Show
    argparser.add_argument('--verbose',default=1,type=int,help='The level of verbose')
    argparser.add_argument('--epoch_show',default=5,type=int,help='The epochs of show time')
    args = argparser.parse_args(args=[])
    return args

args = get_args()

if args.use_gpu:
    device = torch.device('cuda' if torch.cuda.is_available() and args.use_gpu else "cpu")
    
# set path
which_benchmark = 'Syn_'+'_'.join(str(item) for item in [args.sc, args.sh, args.one, args.depX, args.depU,args.VX])
which_dataset = '_'.join(str(item) for item in [args.mV, args.mX, args.mU])
resultDir = args.storage_path + f'/results/{which_benchmark}_{which_dataset}/{args.num}/'
dataDir = f'{args.storage_path}/data/{which_benchmark}/{which_dataset}/'
os.makedirs(os.path.dirname(resultDir), exist_ok=True)
logfile = f'{resultDir}/log.txt'

if args.rewrite_log:
    f = open(logfile,'w')
    f.close()

results = []
for exp in range(args.num_reps):
    # load data
    train_df = pd.read_csv(dataDir + f'{exp}/{args.mode}/train.csv')[:args.num]
    val_df = pd.read_csv(dataDir + f'{exp}/{args.mode}/val.csv')[:args.num]
    test_df = pd.read_csv(dataDir + f'{exp}/{args.mode}/test.csv')[:args.num]

    train = CausalDataset(train_df, variables = ['u','x','v','z','p','s','m','t','g','y','f','c'])
    val = CausalDataset(val_df, variables = ['u','x','v','z','p','s','m','t','g','y','f','c'])
    test = CausalDataset(test_df, variables = ['u','x','v','z','p','s','m','t','g','y','f','c'])

    res_list = []
    
    monitor = run_DFIV(exp, args, dataDir, resultDir, train, val, test, device)
    res_list = res_list + [monitor.t_best[0][0],monitor.t_best[0][1],monitor.t_best[1][0],monitor.t_best[1][1]]
    res_list = res_list + [monitor.y_best[0][0],monitor.y_best[0][1],monitor.y_best[1][0],monitor.y_best[1][1]]
    res_list = res_list + [monitor.final[0][0],monitor.final[0][1],monitor.final[1][0],monitor.final[1][1]]
    
    res = np.array(res_list) - 1.0
    results.append(res)

results.append(np.mean(results,0))
results.append(np.std(results,0))
    
res_df = pd.DataFrame(np.array(results),
                       columns=[ alpha+data_cls for alpha in ['t_best_iv','t_best_t','y_best_iv','y_best_t','final_iv','final_t'] for data_cls in ['_train', '_test']]).round(4)
res_df.to_csv(resultDir + f'DFIV_{args.mode}_result.csv', index=False)
res_df